In [18]:
%pylab inline
pylab.rcParams['figure.figsize'] = (12, 8)

import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import sys; sys.path.append('../')
from PDE_FIND import *
import itertools

Populating the interactive namespace from numpy and matplotlib


In [19]:
tb, te = 0., 15.
xb, xe = -2*np.pi, 2*np.pi
vb, ve = -4.5, 4.5

BD = 2 #boundary cells in x-dimension
mx, mvx = 32, 32
sizex = mx + 2*BD
steps = 150 #number of samples in time 

#load data
path = '../Datasets/VlasovLD/'

fe = np.load(path+'f_e.npy')
x = np.load(path+'x.npy')
v = np.load(path+'v.npy')
E = np.load(path+'E.npy')
t = np.load(path+'t.npy')

In [20]:
print(np.shape(E))
print(np.shape(x))
print(np.shape(v))
print(np.shape(t))
print(np.shape(fe))

(150, 36)
(36,)
(32,)
(150,)
(150, 36, 32)


In [21]:
dx = x[1] - x[0]
dv = v[1] - v[0]
dt = t[1] - t[0]
print(dx)
print(dv)
print(dt)

0.3926990816987237
0.29032258064516103
0.10999999999999999


In [22]:
#compute velocity moments of the distribution function

#first few velocity moments
n = np.zeros((steps, sizex)) 
nu = np.zeros((steps, sizex)) 
nuu = np.zeros((steps, sizex))

for s in range(steps):
    for i in range(sizex):
        n[s,i] = np.sum(dv*fe[s,i,:])
        nu[s,i] = np.sum(dv*fe[s,i,:] * v[:])
        nuu[s,i] = np.sum(dv*fe[s,i,:] * v[:]**2)
            

In [105]:
#construct theta and compute Ut

#take up to second order derivatives
n_t = np.zeros((steps, sizex))

n_x = np.zeros((steps, sizex))
n_xx = np.zeros((steps, sizex))
nu_x = np.zeros((steps, sizex))
nu_xx = np.zeros((steps, sizex))
nuu_x = np.zeros((steps, sizex))
nuu_xx = np.zeros((steps, sizex))


dertype = 'NpGrad' #method used for derivatives (other options: 'PolyD', 'NpGrad', 'FDconv')
#NOTE: PolyD not working yet

#t-derivative
if (dertype == 'FD'):
    for i in range(sizex):
        n_t[:,i] = FiniteDiff(n[:,i], dt, 1)
    
elif (dertype == 'PolyD'):
    for i in range(sizex):
        n_t[:,i] = PolyDiff(n[:,i], t, deg=3, diff=1, width=3)

elif (dertype == 'NpGrad'):
    for i in range(sizex):
        n_t[:,i] = np.gradient(n[:,i], t, edge_order = 2)
        
elif (dertype == 'FDconv'):
    for i in range(sizex):
        smooth = ConvSmoother(n[:,i], p=1, sigma=3*dt)
        n_t[:,i] = FiniteDiff(smooth, dt, 1)
    
#x-derivatives
if (dertype == 'FD'):
    for s in range(steps):
        n_x[s,:] = FiniteDiff(n[s,:], dx, 1)
        n_xx[s,:] = FiniteDiff(n[s,:], dx, 2)
        
        nu_x[s,:] = FiniteDiff(nu[s,:], dx, 1)
        nu_xx[s,:] = FiniteDiff(nu[s,:], dx, 2)
        
        nuu_x[s,:] = FiniteDiff(nuu[s,:], dx, 1)
        nuu_xx[s,:] = FiniteDiff(nuu[s,:], dx, 2)

elif (dertype == 'PolyD'):
    for s in range(steps):
        w = 1
        width = 2*w+1
        n_x[s,w:sizex-w] = PolyDiff(n[s,:], x, deg=3, diff=1, width=3)
        n_xx[s,w:sizex-w] = PolyDiff(n[s,:], x, deg=3, diff=2, width=3)
        
        nu_x[s,w:sizex-w] = PolyDiff(nu[s,:], x, deg=3, diff=1, width=3)
        nu_xx[s,w:sizex-w] = PolyDiff(nu[s,:], x, deg=3, diff=2, width=3)
        
        nuu_x[s,:] = PolyDiff(nuu[s,:], x, deg=3, diff=1, width=3)
        nuu_xx[s,:] = PolyDiff(nuu[s,:], x, deg=3, diff=2, width=3)
        
elif (dertype == 'NpGrad'):
    for s in range(steps):
        n_x[s,:] = np.gradient(n[s,:], x, edge_order=2)
        n_xx[s,:] = np.gradient(n_x[s,:], x, edge_order=2)
        
        nu_x[s,:] = np.gradient(nu[s,:], x, edge_order=2)
        nu_xx[s,:] = np.gradient(nu_x[s,:], x, edge_order=2)
        
        nuu_x[s,:] = np.gradient(nuu[s,:], x, edge_order=2)
        nuu_xx[s,:] = np.gradient(nuu_x[s,:], x, edge_order=2)
        
elif (dertype == 'FDconv'):
    for s in range(steps):
        smooth = ConvSmoother(n[s,:], p=1, sigma=3*dx)
        n_x[s,:] = FiniteDiff(smooth, dx, 1)
        n_xx[s,:] = FiniteDiff(smooth, dx, 2)
        
        smooth = ConvSmoother(nu[s,:], p=1, sigma=3*dx)
        nu_x[s,:] = FiniteDiff(smooth, dx, 1)
        nu_xx[s,:] = FiniteDiff(smooth, dx, 2)
        
        smooth = ConvSmoother(nuu[s,:], p=1, sigma=3*dx)
        nuu_x[s,:] = FiniteDiff(smooth, dx, 1)
        nuu_xx[s,:] = FiniteDiff(smooth, dx, 2)

In [106]:
#additional integration in space/time after the differentiation

integration = True

if integration:

    def simpson(L, R, M):
        return (1./6.)*(L + 4.*M + R)

    def simpson_averaging(L, R, M, dx):
        return (1./dx)*simpson(L, R, M)

    def averaging(arr, w=1):
        global dx, dt

        steps, sizex = np.shape(arr)

        arr_int = np.zeros((steps, sizex))

        domain = 2*w+1

        for s in range(w, steps-w):
            for i in range(w, sizex-w):

                #t averaging
                arr_int[s, i] = simpson_averaging(arr[s-w, i], arr[s+w, i], arr[s, i], domain*dt)

                #x averaging
                arr_int[s, i] = simpson_averaging(arr[s, i-w], arr[s, i+1], arr[s, i], domain*dx)

        #boundaries
        arr_int[0:w, :] = arr[0:w, :]
        arr_int[steps-w:steps,:] = arr[steps-w:steps,:]

        arr_int[:,0:w] = arr[:,0:w]
        arr_int[:,sizex-w] = arr[:,sizex-w]

        return arr_int


    n_int   = averaging(n)
    nu_int  = averaging(nu)
    nuu_int = averaging(nuu)

    n_t_int = averaging(n_t)
    n_x_int = averaging(n_x)
    nu_x_int = averaging(nu_x)
    nuu_x_int = averaging(nuu_x)

    n_xx_int = averaging(n_xx)
    nu_xx_int = averaging(nu_xx)
    nuu_xx_int = averaging(nuu_xx)

    #reshape the data
    num_points = steps * sizex
    print('num_points: ', num_points)

    xn = np.zeros((steps,sizex))
    for s in range(steps):
        xn[s,:] = x[:]

    xn = xn.reshape((num_points, 1))
    nn = n_int.reshape((num_points, 1))
    nun = nu_int.reshape((num_points, 1))
    nuun = nuu_int.reshape((num_points, 1))

    n_t = n_t_int.reshape((num_points, 1))
    n_x = n_x_int.reshape((num_points, 1))
    n_xx = n_xx_int.reshape((num_points, 1))

    nu_x = nu_x_int.reshape((num_points, 1))
    nu_xx = nu_xx_int.reshape((num_points, 1))

    nuu_x = nuu_x_int.reshape((num_points, 1))
    nuu_xx = nuu_xx_int.reshape((num_points, 1))
    
    print('Integration done')

num_points:  5400
Integration done


In [107]:

if (integration != True):
    #reshape the data
    num_points = steps * sizex
    print('num_points: ', num_points)

    xn = np.zeros((steps,sizex))
    for s in range(steps):
        xn[s,:] = x[:]

    xn = xn.reshape((num_points, 1))
    nn = n.reshape((num_points, 1))
    nun = nu.reshape((num_points, 1))
    nuun = nuu.reshape((num_points, 1))

    n_t = n_t.reshape((num_points, 1))
    n_x = n_x.reshape((num_points, 1))
    n_xx = n_xx.reshape((num_points, 1))

    nu_x = nu_x.reshape((num_points, 1))
    nu_xx = nu_xx.reshape((num_points, 1))

    nuu_x = nuu_x.reshape((num_points, 1))
    nuu_xx = nuu_xx.reshape((num_points, 1))
    
    print('No Integration')

In [108]:
#form the data
X_data = np.hstack([nn, nun, nuun, xn])

#X_ders = np.hstack([np.ones((num_points,1)), n_x, nu_x, nuu_x, n_xx, nu_xx, nuu_xx])
#X_ders_descr = ['', 'n_{x}', 'nu_{x}', 'nuu_{x}', 'n_{xx}', 'nu_{xx}', 'nuu_{xx}']

X_ders = np.hstack([np.ones((num_points,1)), n_x, n_xx, nu_x, nu_xx, nuu_x, nuu_xx])
X_ders_descr = ['', 'n_{x}', 'n_{xx}', 'nu_{x}', 'nu_{xx}', 'nuu_{x}', 'nuu_{xx}']

In [109]:
#build theta library
Theta, description = build_Theta(X_data, X_ders, X_ders_descr, 1, data_description=['n', 'nu', 'nuu', 'x'])

print('Candidate terms for PDE')
print(['1'] + description[1:])

Candidate terms for PDE
['1', 'n_{x}', 'n_{xx}', 'nu_{x}', 'nu_{xx}', 'nuu_{x}', 'nuu_{xx}', 'x', 'nuu', 'nu', 'n', 'xn_{x}', 'nuun_{x}', 'nun_{x}', 'nn_{x}', 'xn_{xx}', 'nuun_{xx}', 'nun_{xx}', 'nn_{xx}', 'xnu_{x}', 'nuunu_{x}', 'nunu_{x}', 'nnu_{x}', 'xnu_{xx}', 'nuunu_{xx}', 'nunu_{xx}', 'nnu_{xx}', 'xnuu_{x}', 'nuunuu_{x}', 'nunuu_{x}', 'nnuu_{x}', 'xnuu_{xx}', 'nuunuu_{xx}', 'nunuu_{xx}', 'nnuu_{xx}']


In [110]:
#solve for xi / find sparse solution

#TrainSTRidge
lam  = 10.**-4
d_tol = 0.3

c1 = TrainSTRidge(Theta, n_t, lam, d_tol, maxit=50, print_best_tol=True)

print('Result from TrainSTRidge')
print_pde(c1, description, ut = 'n_{t}')

Optimal tolerance: 0.15667857142857164
Result from TrainSTRidge
n_{t} = (-0.966080 +0.000000i)nu_{x}
   


In [111]:
#STRidge
c2 = STRidge(Theta, n_t, lam=lam,  maxit=50, tol=0.1*d_tol)

print('Result from STRidge')
print_pde(c2, description, ut= 'n_{t}')

Result from STRidge
n_{t} = (-0.966213 +0.000000i)nu_{x}
   


In [117]:
#LASSO
c3 = Lasso(Theta, n_t.flatten(), lam=0.16)

print('Result from Lasso')
print_pde(c3, description, ut='n_{t}')

Result from Lasso
n_{t} = (-1.078938 +0.000000i)nu_{x}
    + (-0.020918 +0.000000i)nuunu_{x}
    + (0.148547 +0.000000i)nnu_{x}
   


In [113]:
#FoBa
c4 = FoBaGreedy(Theta, n_t)

print('Result from FoBaGreedy')
print_pde(c4, description, ut='n_{t}')

Result from FoBaGreedy
n_{t} = (-0.966213 +0.000000i)nu_{x}
   
